# Getting Started

The `opinf` package constructs reduced-order models for large dynamical systems.
Such systems often arise from the numerical solution of partial differentials equations.
In this introductory tutorial, we use operator inference (OpInf) to learn a reduced-order model for a simple heat equation.
This is a simplified version of the first numerical example in {cite}`peherstorfer2016opinf`.

## Problem Statement

:::{admonition} Governing Equations
:class: attention

For the spatial domain $\Omega = [0,L]\subset \RR$ and the time domain $[t_0,t_f]\subset\RR$, consider the one-dimensional heat equation with homogeneous Dirichlet boundary conditions:

\begin{align*}
    &\frac{\partial}{\partial t} q(x,t) = \frac{\partial^2}{\partial x^2}q(x,t)
    & x &\in\Omega,\quad t\in(t_0,t_f],
    \\
    &q(0,t) = q(L,t) = 0
    & t &\in [t_0,t_f],
    \\
    &q(x,t_0) = q_{0}(x)
    & x &\in \Omega.
\end{align*}

This is a model for a one-dimensional rod that conducts heat.
The unknown state variable $q(x,t)$ represents the temperature of the rod at location $x$ and time $t$; the temperature at the ends of the rod are fixed at $0$ and heat is allowed to flow out of the rod at the ends.
:::

:::{admonition} Objective
:class: attention

Construct a low-dimensional system of ordinary differential equations, called the _reduced-order model_ (ROM), which can be solved rapidly to produce approximate solutions $q(x, t)$ to the partial differential equation given above. We will use OpInf to learn the ROM from high-fidelity data for one choice of initial condition $q_0(x)$ and test its performance on new initial conditions.
:::

## Training Data

We begin by generating training data through a traditional finite difference discretization of the PDE.

:::{important}
One key advantage of OpInf is that, because it learns a ROM from data alone, direct access to a high-fidelity solver is not required.
In this tutorial, we explicitly construct the high-fidelity solver, but in practice, we only need the following:
1. Solution outputs of a high-fidelity solver to learn from, and
2. Some knowledge of the structure of the governing equations.
:::

### Define the Full-order Model

To solve the problem numerically, let $\{x\}_{i=0}^{n+1}$ be an equidistant grid of $n+2$ points on $\Omega$, i.e.,

$$
\begin{aligned}
    0 &= x_0 < x_1 < \cdots < x_n < x_{n+1} = L
    &
    &\text{and}
    &
    \delta x &= \frac{L}{n+1} = x_{i+1} - x_{i},\quad i=1,\ldots,n-1.
\end{aligned}
$$

The boundary conditions prescribe $q(x_0,t) = q(x_{n+1},t) = 0$.
Our goal is to compute $q(x, t)$ at the interior spatial points $x_{1}, x_{2}, \ldots, x_{n}$ for various $t = [0,T]$. we wish to compute the state vector

$$
\begin{aligned}
    \q(t)
    = \left[\begin{array}{c}
        q(x_1,t) \\ \vdots \\ q(x_n,t)
    \end{array}\right]\in\RR^n
\end{aligned}
$$

for $t\in[t_0,t_f]$.

Introducing a central finite difference approximation for the spatial derivative,

$$
\begin{aligned}
    \frac{\partial^2}{\partial x^2}q(x,t)
    &\approx \frac{q(x-\delta x,t) - 2q(x,t) + q(x+\delta x,t)}{(\delta x)^2},
\end{aligned}
$$

yields the semi-discrete linear system

$$
\begin{aligned}
    \ddt\q(t) = \A\q(t),
    \qquad
    \q(0) = \q_0,
\end{aligned}
$$ (eq_basics_fom)

where

$$
\begin{aligned}
    \A &= \frac{1}{(\delta x)^2}\left[\begin{array}{ccccc}
        -2 & 1 & & & \\
        1 & -2 & 1 & & \\
        & \ddots & \ddots & \ddots & \\
        & & 1 & -2 & 1 \\
        & & & 1 & -2 \\
    \end{array}\right] \in\RR^{n\times n},
    &
    \q_0 &= \left[\begin{array}{c}
    q_{0}(x_{1}) \\ q_{0}(x_{2}) \\ \vdots \\ q_{0}(x_{n-1}) \\ q_{0}(x_{n})
    \end{array}\right] \in\RR^{n}.
\end{aligned}
$$

Equation {eq}`eq_basics_fom` is called the _full-order model_ (FOM) or the _high-fidelity model_. The computational complexity of solving {eq}`eq_basics_fom` depends on the dimension $n$, which must often be large in order for $\q(t)$ to approximate $q(x,t)$ well over the spatial grid. Our goal is to construct a ROM that approximates the FOM, but whose computational complexity only depends on some smaller dimension $r \ll n$.

### Solve the Full-order Model

For this demo, we'll use $t_0 = 0$ and $L = t_f = 1$.
We begin by simulating the full-order system described above with the initial condition

$$
\begin{aligned}
    q_{0}(x) = x(1 - x),
\end{aligned}
$$

using a maximal time step size $\delta t = 10^{-3}$.
This results in $k = 10^3 + 1 = 1001$ state snapshots (1000 time steps after the initial condition), which are organized as the _snapshot matrix_ $\Q\in\RR^{n\times k}$, where the $j$th column is the solution trajectory at time $t_j$:

$$
\begin{aligned}
    \Q = \left[\begin{array}{ccc}
        && \\
        \q_{0} & \cdots & \q_{k-1}
        \\ &&
    \end{array}\right] \in\RR^{n\times k},
    \qquad
    \q_{j} := \q(t_j) \in\RR^{n},\quad j = 0, \ldots, k-1.
\end{aligned}
$$

Note that the initial condition $\q_{0}$ is included as a column in the snapshot matrix.

In [ ]:
import numpy as np
import pandas as pd
import scipy.linalg as la
import scipy.sparse as sparse
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

import opinf

opinf.utils.mpl_config()

In [ ]:
# Construct the spatial domain.
L = 1  # Spatial domain length.
n = 2**7 - 1  # Spatial grid size.
x_all = np.linspace(0, L, n + 2)  # Full spatial grid.
x = x_all[1:-1]  # Interior spatial grid (where q is unknown).
dx = x[1] - x[0]  # Spatial resolution.

# Construct the temporal domain.
t0, tf = 0, 1  # Initial and final time.
k = tf * 1000 + 1  # Temporal grid size.
t = np.linspace(t0, tf, k)  # Temporal grid.
dt = t[1] - t[0]  # Temporal resolution.

print(f"Spatial step size:\tdx = {dx}")
print(f"Temporal step size:\tdt = {dt}")

In [ ]:
# Construct the full-order state matrix A.
diags = np.array([1, -2, 1]) / (dx**2)
A = sparse.diags(diags, [-1, 0, 1], (n, n))


# Define the full-order model dx/dt = f(t,x),  x(0) = x0.
def fom(t, x):
    return A @ x


# Construct the initial condition for the training data.
q0 = x * (1 - x)

print(f"{A.shape=}\t{q0.shape=}")

In [ ]:
# Compute snapshots by solving the full-order model with SciPy.
Q = solve_ivp(fom, [t0, tf], q0, t_eval=t, method="BDF").y

print(f"{Q.shape=}")

:::{caution}
It is often better to use your own ODE solver, tailored to the problem at hand, instead of integration packages such as [**scipy.integrate**](https://docs.scipy.org/doc/scipy/tutorial/integrate.html).
If the integration strategy of the FOM is known, try using that strategy with the ROM.
:::

### Visualize Training Data

Next, we visualize the snapshots to get a sense of how the solution looks qualitatively.

In [ ]:
def plot_heat_data(Z, title, ax=None):
    """Visualize temperature data in space and time."""
    if ax is None:
        _, ax = plt.subplots(1, 1)

    # Plot a few snapshots over the spatial domain.
    sample_columns = [0, 2, 5, 10, 20, 40, 80, 160, 320]
    color = iter(plt.cm.viridis_r(np.linspace(0.05, 1, len(sample_columns))))

    leftBC, rightBC = [0], [0]
    for j in sample_columns:
        q_all = np.concatenate([leftBC, Z[:, j], rightBC])
        ax.plot(x_all, q_all, color=next(color), label=rf"$q(x,t_{{{j}}})$")

    ax.set_xlim(x_all[0], x_all[-1])
    ax.set_xlabel(r"$x$")
    ax.set_ylabel(r"$q(x,t)$")
    ax.legend(loc=(1.05, 0.05))
    ax.set_title(title)

In [ ]:
plot_heat_data(Q, "Snapshot data")

This matches our intuition: initially there is more heat toward the center of the rod, which then diffuses out of the ends of the rod. In the figure, earlier times are lighter colors and later times are darker colors.

## Operator Inference

At this point, we have gathered some training data by simulating the FOM.
We also have an initial condition and space and time domains.

| Name | Symbol | Code Variable |
| :--- | :----: | :------------ |
| State snapshots | $\Q$ | `Q` |
| Initial state | $\q_0$ | `q0` |
| Spatial variable | $\Omega$ | `x` |
| Time domain | $[t_0,t_f]$ | `t` |

Our task now is to construct a low-dimensional system whose solutions can be used as approximate solutions to the PDE.
Below we show the overall process, then explain each piece in detail.

In [ ]:
import opinf

# Define the reduced-order model structure.
rom = opinf.ROM(
    basis=opinf.basis.PODBasis(cumulative_energy=0.999999),
    ddt_estimator=opinf.ddt.UniformFiniteDifferencer(t, "ord6"),
    model=opinf.models.ContinuousModel("A"),
    solver=opinf.lstsq.L2Solver(regularizer=1e-2),
)

# Calibrate the reduced-order model to data.
rom.fit(Q)

# Solve the reduced-order model.
Q_ROM = rom.predict(q0, t, method="BDF", max_step=dt)

# Compute the relative error of the ROM solution.
opinf.post.frobenius_error(Q, Q_ROM)[1]

### Data Compression

Our first task is to construct a low-dimensional representation of the FOM state $\q(t)\in\RR^{n}$, denoted $\qhat(t)\in\RR^r$.
A ROM is a system of equations that acts on the reduced state $\qhat(t)$.
The integer $r$ is the dimension of the ROM: if $r \ll n$, we can expect to be able to solve the ROM much faster than we can solve the FOM.

The relationship between $\q(t)$ and $\qhat(t)$ helps dictate the structure of the ROM and allows us to compress the state snapshots $\q_0,\ldots,\q_{k-1}\in\RR^{n}$ to low-dimensional representations $\qhat_0,\ldots,\qhat_{k-1}\in\RR^{r}$ that are used to calibrate the ROM.
Tools for defining low-dimensional approximations of high-dimensional states are defined in {mod}`opinf.basis`.

For this problem, we use a linear approximation for $\q(t)$:

$$
\begin{aligned}
    \q(t)
    \approx \Vr\qhat(t).
\end{aligned}
$$

The matrix $\Vr\in\RR^{n\times r}$ is called a _basis matrix_ and its columns are called _basis vectors_.
We typically have $\Vr\trp\Vr = \I$, i.e., the basis vectors form an orthonormal set.
Note that the product $\Vr\qhat(t)$ is a linear combination of the basis vectors, so $\q(t)$ can only be approximated well if it is within or near the span of the basis vectors.

We choose $\Vr$ using proper orthogonal decomposition (POD), which is based on the singular value decomposition (SVD) of samples of $\q(t)$.
The singular values give some guidance on choosing an appropriate ROM dimension $r$.
Fast singular value decay is a good sign that a ROM may be successful with this kind of data; if the singular values do not decay quickly, then a large $r$ may be required to capture the behavior of the system.
POD is implemented in this package as {class}`opinf.basis.PODBasis`.
Below, we initialize a `PODBasis` with a criteria for selecting $r$: choose the smallest $r$ such that we capture over $99.9999\%$ of the [cumulative energy](#sec:api-basis-dimselect) of the system.


In [ ]:
# Initialize a basis.
basis = opinf.basis.PODBasis(cumulative_energy=0.999999)

# Fit the basis (compute Vr) using the snapshot data.
basis.fit(Q)
print(basis)

# Visualize the basis vectors.
basis.plot1D(x)
plt.show()

Solutions of our eventual ROM are restricted to linear combinations of these two basis vectors.

After the `PODbasis` is initialized and calibrated, we can use it to compress the state snapshots to an $r$-dimensional representation.
In this case, we have $\qhat_j = \Vr\trp\q_j \in \RR^{r}$.
These $\qhat_j$ are data for the ROM state $\qhat(t)$ at time $t_j$.

In [ ]:
# Compress the state snapshots to the reduced space defined by the basis.
Q_ = basis.compress(Q)

print(f"{Q.shape=}, {Q_.shape=}")

To see how well the state can be represented by a given basis matrix, it is helpful to examine the _projection_ of the state snapshots.
For linear state approximations, the projection of $\q\in\RR^n$ is the vector $\Vr\Vr\trp\q\in\RR^n$.

In [ ]:
basis.projection_error(Q)

### Time Derivative Estimation

In addition to the compressed state snapshots $\qhat_0,\ldots,\qhat_{k-1}$, OpInf for time-continuous (ODE) models requires data for the time derivatives of the state snapshots, denoted

$$
\begin{aligned}
    \dot{\qhat}_j
    = \ddt\qhat(t)\big|_{t=t_j}
    \in\RR^{r}.
\end{aligned}
$$

There are two ways to get such data.

1. If time derivatives of the original state snapshots are available, they can be compressed to the reduced state space.
2. Otherwise, the time derivatives may be estimated from the compressed states.

The latter scenario (being given state data but not time derivative data) is common, so {mod}`opinf.ddt` defines tools for estimating time derivatives.

Recall that the FOM in this problem is given by $\ddt\q(t) = \A\q(t)$.
In this case we have $\A$, so we can compute $\dot{\q}_j = \A\q_j$, then set $\dot{\qhat}_j = \Vr\trp\dot{\q}_j$.
Below, we should how this approach compares with using tools from {mod}`opinf.ddt`.
Since the data $\q_0,\ldots,\q_{k-1}$ are defined on a uniform time grid, we use {class}`opinf.ddt.UniformFiniteDifferencer`.

In [ ]:
# Compute exact time derivatives using the FOM and compress them.
Qdot_exact = basis.compress(A @ Q)

# Estimate time derivatives using 6th-order finite differences.
ddt_estimator = opinf.ddt.UniformFiniteDifferencer(t, "ord6")
Qdot_ = ddt_estimator.estimate(Q_)[1]

print(f"{Qdot_exact.shape=}\t{Qdot_.shape=}")

In [ ]:
# Check that the estimate is close to the true time derivatives.
la.norm(Qdot_exact - Qdot_, ord=np.inf) / la.norm(Qdot_exact, ord=np.inf)

### Specifying the Model Form

We now have low-dimensional state and time derivative data.
To learn a ROM with OpInf, we must specify the structure of the ROM, which should be motivated by the FOM and the dimensionality reduction strategy.

The FOM is a linear system of ODEs,

$$
\begin{aligned}
    \ddt\q(t) = \A\q(t),
    \qquad
    \q(0) = \q_0.
\end{aligned}
$$

Substituting in the approximation $\q(t)\approx\Vr\qhat(t)$, we have

$$
\begin{aligned}
    \ddt\Vr\qhat(t) = \A\Vr\qhat(t),
    \qquad
    \Vr\qhat(0) = \q_0.
\end{aligned}
$$

Next, left multiply by $\Vr\trp$ and use the fact that $\Vr\trp\Vr = \I$ to get the following:

$$
\begin{aligned}
    \ddt\qhat(t) = \tilde{\A}\qhat(t),
    \qquad
    \qhat(0) = \Vr\trp\q_0,
\end{aligned}
$$

where $\tilde{\A} = \Vr\trp\A\Vr \in \RR^{r\times r}$.
This is called the _intrusive Galerkin ROM_ corresponding to the FOM and the choice of basis matrix $\Vr$.
The intrusive ROM can only be constructed if $\A$ is known; with OpInf, we construct a reduced system with the same linear structure as the intrusive ROM, but without using $\A$ explicitly:

$$
\begin{aligned}
    \ddt\qhat(t) = \Ahat\qhat(t),
    \qquad
    \qhat(0) = \Vr\trp\q_0,
\end{aligned}
$$

where $\Ahat\in\RR^{r\times r}$.
We specify this linear structure by initializing an {class}`opinf.models.ContinuousModel` with the string `"A"`.


In [ ]:
model = opinf.models.ContinuousModel("A")
print(model)

:::{tip}
The `"A"` syntax is a shortcut for a slightly longer statement:

```python
>>> model = opinf.models.ContinuousModel([opinf.operators.LinearOperator()])
```

The {class}`opinf.operators.LinearOperator` class represents the $r \times r$ matrix $\Ahat$, whose entries will be calibrated via regression.
See {mod}`opinf.operators` for the kinds of terms that OpInf ROMs can contain.
:::

### Calibrating Model Operators

Our task now is to learn the entries of $\Ahat$ using the compressed state snapshots $\qhat_0,\ldots,\qhat_{k-1}$ and the corresponding time derivatives $\dot{\qhat}_0,\ldots,\dot{\qhat}_{k-1}$.
OpInf does this through minimizing the residual of the model equation with respect to the data:

$$
    \min_{\Ahat\in\RR^{r\times r}}\sum_{j=0}^{k-1}\left\|
        \Ahat\Vr\trp\q_{j} - \Vr\trp\dot{\q}_{j}
    \right\|_{2}^2
    + \mathcal{R}(\Ahat),
$$ (eq_basics_opinf)

where $\mathcal{R}(\Ahat)$ is a regularization term (more on this soon).
The {mod}`opinf.lstsq` module defines tools for solving this problem (or variations on it).

By default, the regression is solved without regularization, i.e., $\mathcal{R}(\Ahat) = 0$.
The following code compares the OpInf ROM matrix $\Ahat$ to the intrusive ROM matrix $\tilde{\A} = \Vr\trp\A\Vr$.

In [ ]:
# Construct the intrusive ROM linear operator.
Vr = basis.entries
A_intrusive = Vr.T @ A @ Vr

# Construct the OpInf ROM and extract the linear operator.
model.fit(states=Q_, ddts=Qdot_exact)
A_opinf = model.operators[0].entries

# Compare the two linear operators.
np.allclose(A_intrusive, A_opinf)

In this simple problem, using exact time derivative data $\Vr\trp\A\Q$ and with zero regularization, OpInf produces the intrusive ROM.
However, if estimated time derivative data are used instead, the OpInf ROM differs slightly from the intrusive ROM.

In [ ]:
# Construct the OpInf ROM with estimated time derivatives.
model.fit(states=Q_, ddts=Qdot_)
A_opinf = model.operators[0].entries

np.allclose(A_intrusive, A_opinf)

In [ ]:
# Check the difference between intrusive projection and OpInf.
la.norm(A_intrusive - A_opinf) / la.norm(A_intrusive)

### Regularization: Stabilizing the Inference Problem


Ill-conditioning in the data, errors in the estimation of the time derivatives, or overfitting to the data can result in an $\Ahat$ that defines an inaccurate or even unstable ROM.
Introducing a regularization term promotes solutions that respect both the training data and the physics of the problem.
One common option is [Tikhonov regularization](https://en.wikipedia.org/wiki/Tikhonov_regularization), which sets $\mathcal{R}(\Ahat) = \|\lambda\Ahat\|_{F}^{2}$ to penalize the entries of the learned operators.


In [ ]:
# Define a solver for the Tikhonov-regularized least-squares problem.
solver = opinf.lstsq.L2Solver(regularizer=1e-2)

# Construct the OpInf ROM through regularized least squares.
model.fit(states=Q_, ddts=Qdot_, solver=solver)
A_opinf = model.operators[0].entries

# Compare to the intrusive model.
np.allclose(A_intrusive, A_opinf)

In [ ]:
# Check the difference between intrusive projection and OpInf.
la.norm(A_intrusive - A_opinf) / la.norm(A_intrusive)

:::{note}
With inexact time derivatives or regularization, OpInf differs from the intrusive operator $\tilde{\A}$.
However, we will see that the ROM produced by OpInf is highly accurate.
In fact, it is sometimes the case that OpInf outperforms intrusive projection.
:::


:::{important}
Regularization is important in all but the simplest OpInf problems.
If OpInf produces an unstable ROM, try different values for the `regularizer`.
See {cite}`mcquarrie2021combustion` for an example of a principled choice of regularization for a combustion problem.
:::

### Solving the Reduced-order Model

Once the model is calibrated, we may solve the ROM with {meth}`opinf.models.ContinuousModel.predict`, which wraps [**scipy.integrate.solve_ivp()**](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html).
This method takes an initial condition for the model $\qhat_0 = \Vr\trp\q_0$, the time domain over which to record the solution, and any additional arguments for the integrator.

In [ ]:
q0_ = basis.compress(q0)  # Compress the initial conditions.

Q_ROM_ = model.predict(q0_, t, method="BDF")

print(f"{Q_ROM_.shape=}")

The solution is still in the low-dimensional state space; it can be mapped to the original state space by applying $\Vr$.

In [ ]:
Q_ROM = basis.decompress(Q_ROM_)

print(f"{Q_ROM.shape=}")

:::{tip}
{meth}`opinf.models.ContinuousModel.predict` is convenient, but [**scipy.integrate.solve_ivp()**](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) implements relatively few time integration schemes.
However, the ROM can be simulated by **any** ODE solver scheme by extracting the inferred operator $\Ahat$. 
If `solver(A, q0)` were a solver for systems of the form $\ddt\qhat = \Ahat\qhat(t),\ \qhat(0) = \qhat_0$, we could simulate the ROM with the following code.

```python
q0_ = Vr.T @ q0                           # Compress the initial conditions.
Q_ROM_ = solver(model.A_.entries, q0_)    # Solve the ROM in the reduced space.
Q_ROM = Vr @ Q_ROM_                       # Decompress the ROM solutions.
```

More generally, the method {meth}`opinf.models.ContinuousModel.rhs` represents the right-hand side of the model, the $\hat{\mathbf{f}}$ of $\ddt\qhat(t) = \hat{\mathbf{f}}(t, \qhat(t))$.
General-purpose integrators can therefore be applied to the function {meth}`opinf.models.ContinuousModel.rhs`.
:::

### The ROM Class

Up to this point, we have done the following steps using several package submodules.

1. {mod}`opinf.basis`: Data compression.
2. {mod}`opinf.ddt`: Time derivative estimation.
3. {mod}`opinf.models`: Specify model form, calibrate model operators, solve reduced system.

The [`opinf.ROM`](opinf.roms.ROM) class wraps these steps for convenience.
Its constructor takes a initialized basis, time derivative estimator, and model objects.
Then, [`fit()`](opinf.roms.ROM.fit) calibrates the basis, compresses the state data, estimates the time derivatives, and calibrates the model.
Use [`predict()`](opinf.roms.ROM.predict) to compress initial conditions, solve the model, and express the solutions in the original state space.

In [ ]:
rom = opinf.ROM(
    basis=opinf.basis.PODBasis(cumulative_energy=0.999999),
    ddt_estimator=opinf.ddt.UniformFiniteDifferencer(t, "ord6"),
    model=opinf.models.ContinuousModel("A"),
    solver=opinf.lstsq.L2Solver(regularizer=1e-2),
)

rom.fit(Q)

Q_ROM_2 = rom.predict(q0, t, method="BDF")

np.all(Q_ROM_2 == Q_ROM)

### Evaluate ROM Performance

To get a sense of how well the ROM approximates the FOM, we begin by visualizing the simulation output `Q_ROM`.
It should look similar to the plot of the snapshot data `Q`.

In [ ]:
fig, [ax1, ax2] = plt.subplots(1, 2)
plot_heat_data(Q, "Snapshot data", ax1)
plot_heat_data(Q_ROM, "ROM state output", ax2)
ax1.legend([])
plt.show()

For more detail, we evaluate the $\ell^2$ error of the ROM output in time, comparing it to the snapshot set via {func}`opinf.post.lp_error`.

In [ ]:
abs_l2err, rel_l2err = opinf.post.lp_error(Q, Q_ROM)
plt.semilogy(t, abs_l2err)
plt.title(r"Absolute $\ell^{2}$ error")
plt.show()

In this simple example, the error decreases with time (as solutions get quickly pushed to zero), but this is not the kind of error behavior that should be expected for less trivial systems.

We can also get a scalar error measurement by calculating the relative Frobenius norm error with {func}`opinf.post.frobenius_error`.

In [ ]:
abs_froerr, rel_froerr = opinf.post.frobenius_error(Q, Q_ROM)
print(f"Relative Frobenius-norm error: {rel_froerr:%}")

In other words, the ROM simulation is within 0.1% of the snapshot data.
Note that this value is very close to the projection error that we calculated earlier.

## Prediction: New Initial Conditions

The ROM was trained using only data corresponding to the initial condition $q_0(x) = x(1 - x).$ We'll now test the ROM on the following new initial conditions and compare the results to the corresponding FOM solution:

\begin{align*}
    q_0(x) &= 10x (1 - x),
    &
    q_0(x) &= x^{2}(1 - x)^{2},
    \\
    q_0(x) &= x^{4}(1 - x)^{4},
    &
    q_0(x) &= \sqrt{x(1 - x)},
    \\
    q_0(x) &= \sqrt[4]{x(1 - x)},
    &
    q_0(x) &= \sin(\pi x) + \tfrac{1}{5}\sin(5\pi x).
\end{align*}

Before we compute the ROM error, we also compute the _projection error_ of the new initial condition,

$$
    \frac{||\q_{0} - \Vr \Vr\trp\q_{0}||_{2}}{||\q_{0}||_{2}}.
$$

If this projection error is large, then the new initial condition cannot be represented well within the range of $\Vr$. This will be apparent in the ROM solutions.

### First Attempt

In [ ]:
def test_new_initial_condition(q0, rom, label=None):
    """Compare full-order model and reduced-order model solutions for a given
    initial condition.

    Parameters
    ----------
    q0 : (n,) ndarray
        Heat equation initial conditions q0(x) to be tested.
    rom : opinf.ROM
        Trained reduced-order model object.
    label : str
        LaTeX description of the initial condition being tested.
    """
    # Calculate the projection error of the new initial condition.
    rel_projerr = rom.basis.projection_error(q0, relative=True)

    # Solve the full-order model (FOM) and the reduced-order model (ROM).
    Q_FOM = solve_ivp(fom, [t0, tf], q0, t_eval=t, method="BDF").y
    Q_ROM = rom.predict(q0, t, method="BDF")

    # Plot the FOM and ROM solutions side by side.
    fig, [ax1, ax2] = plt.subplots(1, 2)
    plot_heat_data(Q_FOM, "Full-order model solution", ax1)
    plot_heat_data(Q_ROM, "Reduced-order model solution", ax2)
    ax1.legend([])
    if label:
        fig.suptitle(label, y=1)
    fig.tight_layout()

    # Calculate the ROM error in the Frobenius norm.
    abs_froerr, rel_froerr = opinf.post.frobenius_error(Q_FOM, Q_ROM)

    # Report results.
    plt.show()
    print(
        f"Relative projection error of initial condition: {rel_projerr:.2%}",
        f"Relative Frobenius-norm ROM error: {rel_froerr:.2%}",
        sep="\n",
    )
    return rel_projerr, rel_froerr

In [ ]:
q0_new = [
    10 * x * (1 - x),
    x**2 * (1 - x) ** 2,
    x**4 * (1 - x) ** 4,
    np.sqrt(x * (1 - x)),
    np.sqrt(np.sqrt(x * (1 - x))),
    np.sin(np.pi * x) + np.sin(5 * np.pi * x) / 5,
]

q0_titles = [
    r"$q_{0}(x) = 10 x (1 - x)$",
    r"$q_{0}(x) = x^{2} (1 - x)^{2}$",
    r"$q_{0}(x) = x^{4} (1 - x)^{4}$",
    r"$q_{0}(x) = \sqrt{x (1 - x)}$",
    r"$q_{0}(x) = \sqrt[4]{x (1 - x)}$",
    r"$q_{0}(x) = \sin(\pi x) + \frac{1}{5}\sin(5\pi x)$",
]

results = {}
for i, [q00, title] in enumerate(zip(q0_new, q0_titles)):
    results[f"Experiment {i+1:d}"] = test_new_initial_condition(
        q00, rom, f"Experiment {i+1}: {title}"
    )

labels = [
    "Relative projection error of initial condition",
    "Relative Frobenius-norm ROM error",
]
pd.DataFrame(results, index=labels).T

### Second Attempt: a Better Basis

The ROM performs well for $q_{0}(x) = 10x(1 - x)$, which is unsurprising because this new initial condition is a scalar multiple of the initial condition used to generate the training data.
In other cases, the ROM is less successful because the new initial condition cannot be represented well in the span of the basis vectors.
For example:

In [ ]:
def plot_initial_condition_projection(base):
    """Plot initial conditions 4 and 5 and their projections with respect to
    the basis `base`.

    Parameters
    ----------
    base : opinf.basis.PODBasis
        Trained basis object.
    """
    fig, axes = plt.subplots(1, 2)
    for j, ax in zip([4, 5], axes):
        ax.plot(
            x,
            q0_new[j],
            label=r"True initial condition ($\mathbf{q}_{0}$)",
        )
        ax.plot(
            x,
            base.project(q0_new[j]),
            "--",
            label=r"Basis approximation of initial condition "
            r"($\mathbf{V}_{\!r}\mathbf{V}_{\!r}^{\mathsf{T}}\mathbf{q}_{0}$)",
        )
        ax.set_title(f"Experiment {j+1:d}")

    fig.tight_layout(rect=[0, 0.15, 1, 1])
    axes[0].legend(
        loc="lower center",
        fontsize="large",
        bbox_to_anchor=(0.5, -0.05),
        bbox_transform=fig.transFigure,
    )
    plt.show()

In [ ]:
plot_initial_condition_projection(basis)

To improve the ROM performace _without getting new data from the FOM_, we will enrich the basis by

1. Including the new initial conditions in the basis computation, and 
2. Using a few more basis vectors (we currently have $r = 2$, let's use $r = 5$).

In [ ]:
# Include the new initial conditions in the basis training data.
Q_and_new_q0s = np.column_stack((Q, *q0_new))
newbasis = opinf.basis.PODBasis(num_vectors=5).fit(Q_and_new_q0s)
print(newbasis)

# Plot the projection of the initial conditions in the new basis
plot_initial_condition_projection(newbasis)

In [ ]:
# Initialize a ROM with the new basis.
rom = opinf.ROM(
    basis=newbasis,
    ddt_estimator=opinf.ddt.UniformFiniteDifferencer(t, "ord6"),
    model=opinf.models.ContinuousModel("A"),
    solver=opinf.lstsq.L2Solver(regularizer=1e-5),
)

# Use the same training data as before, but do not reset the basis.
_ = rom.fit(Q, fit_basis=False)

In [ ]:
# Repeat the experiments.
results_new = {}
for i, [q00, title] in enumerate(zip(q0_new, q0_titles)):
    results_new[f"Experiment {i+1:d}"] = test_new_initial_condition(
        q00,
        rom,
        f"Experiment {i+1}: {title}",
    )

# Display results summary.
pd.DataFrame(results_new, index=labels).T

With a more expressive basis, we are now capturing the true solutions with the ROM to within 1% error in the Frobenius norm.

:::{admonition} Takeaway
:class: attention
This example illustrates a fundamental principle of model reduction: the accuracy of the ROM is limited by the accuracy of the underlying low-dimensional approximation, which in this case is $\q(t) \approx \Vr\qhat(t)$. In other words, a good $\Vr$ is critical in order for the ROM to be accurate and predictive.
:::